In [5]:
import numpy as np
import pandas as pd
import math
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Master Data Sets.ipynb")
from matplotlib import pyplot as plt
from ast import literal_eval
import sklearn.metrics.pairwise as sk
# from sklearn.metrics.pairwise import haversine_distances

In [6]:
haversine_distances

NameError: name 'haversine_distances' is not defined

In [2]:
train_cust =pd.read_csv('train_customers.csv',parse_dates=['created_at','updated_at'])
test_cust =pd.read_csv('test_customers.csv',parse_dates=['created_at','updated_at'])
train_loc =pd.read_csv('train_locations.csv')
test_loc =pd.read_csv('test_locations.csv')
vendors =pd.read_csv('vendors.csv',parse_dates=['created_at','updated_at'])
orders =pd.read_csv('orders.csv',parse_dates=['created_at','delivery_date'])
submission =pd.read_csv('SampleSubmission.csv')
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", 200)

/Users/apple/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (15,16,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def haversine(lat1, lon1, lat2, lon2,to_radians=True, earth_radius=6371):
    

    #if to_radians:
        #lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    
    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


In [4]:
dist1=train_loc[['customer_id','location_number','latitude','longitude']]
dist2=vendors[['id','latitude','longitude']]
dist=dist1.assign(key=1).merge(dist2.assign(key=1), on='key').drop('key',axis=1)

In [5]:
dist['distance'] = \
    haversine(dist['latitude_x'], dist['longitude_x'], dist['latitude_y'], dist['longitude_y'])

In [6]:
dist

,customer_id,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,distance
0,02SFNJH,0,1.682392,-78.789737,4,-0.588596,0.754434,13358.377065
1,02SFNJH,0,1.682392,-78.789737,13,-0.471654,0.744470,12611.713000
2,02SFNJH,0,1.682392,-78.789737,20,-0.407527,0.643681,12148.837331
3,02SFNJH,0,1.682392,-78.789737,23,-0.585385,0.753811,13337.678702
4,02SFNJH,0,1.682392,-78.789737,28,0.480602,0.552850,6463.420373
...,...,...,...,...,...,...,...,...
5950295,HWELAU8,1,-0.067043,0.648221,849,-1.588060,-0.066441,9663.509784
5950296,HWELAU8,1,-0.067043,0.648221,855,2.145206,0.745025,14074.079764
5950297,HWELAU8,1,-0.067043,0.648221,856,0.251469,0.483632,2280.034782
5950298,HWELAU8,1,-0.067043,0.648221,858,0.019817,0.587087,676.576771


In [7]:
dist_des=dist['distance'].value_counts(bins =10).rename_axis('distance').reset_index(name='counts')
dist_des.sort_values(by='distance', ascending=True)

,distance,counts
5,"(-20.007, 2000.602]",525586
1,"(2000.602, 4001.204]",926534
0,"(4001.204, 6001.807]",1029965
3,"(6001.807, 8002.409]",634707
7,"(8002.409, 10003.011]",458097
6,"(10003.011, 12003.613]",460579
4,"(12003.613, 14004.215]",572904
2,"(14004.215, 16004.818]",770581
8,"(16004.818, 18005.42]",442178
9,"(18005.42, 20006.022]",128569


In [8]:
dist.describe()

,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,distance
count,5.950300e+06,5.949700e+06,5.949700e+06,5.950300e+06,5.950300e+06,5.950300e+06,5.949700e+06
mean,7.535923e-01,2.361135e+00,-2.511541e+01,2.888400e+02,1.875505e+00,8.036817e-01,8.582728e+03
std,1.355634e+00,2.273392e+01,3.694571e+01,2.410855e+02,2.049760e+01,4.375815e+00,5.227529e+03
min,0.000000e+00,-1.038149e+03,-1.344153e+02,4.000000e+00,-1.787413e+00,-1.449556e+00,3.083878e-10
25%,0.000000e+00,-5.025926e-01,-7.852257e+01,1.057500e+02,-8.091241e-01,8.161790e-02,4.065162e+03
50%,0.000000e+00,-8.785962e-02,2.134231e-02,2.020000e+02,-4.934212e-01,5.271227e-01,7.500168e+03
75%,1.000000e+00,2.617273e-01,5.146713e-01,3.872500e+02,1.591538e-02,6.447635e-01,1.357060e+04
max,2.900000e+01,4.536538e+02,4.535423e+01,9.070000e+02,2.052423e+02,4.421081e+01,2.000602e+04


In [9]:
dist.sort_values(by='distance', ascending=True)

,customer_id,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,distance
4702470,ADL2P50,1,0.005748,0.482281,303,0.005748,0.482281,3.083878e-10
4412682,VVWLRL0,0,2.015615,0.749220,573,2.015615,0.749220,3.980993e-10
2976788,ZGFSYCZ,13,-0.116173,0.570707,676,-0.116173,0.570707,4.088872e-10
2799742,3CJKSR9,6,-0.060056,0.652652,191,-0.060048,0.652630,1.514891e-01
5267974,3T31BBH,0,-1.002260,0.078609,386,-1.002137,0.078609,7.846886e-01
...,...,...,...,...,...,...,...,...
5919895,O0LALCF,0,NaN,NaN,849,-1.588060,-0.066441,NaN
5919896,O0LALCF,0,NaN,NaN,855,2.145206,0.745025,NaN
5919897,O0LALCF,0,NaN,NaN,856,0.251469,0.483632,NaN
5919898,O0LALCF,0,NaN,NaN,858,0.019817,0.587087,NaN


In [10]:
dist4=dist[(dist['distance']>=0)&(dist['distance']<=15)]
dist4

,customer_id,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,distance
7292,IHVLOYG,1,-1.269978,0.088583,843,-1.269317,0.082343,12.521788
68511,KXWONMC,0,-0.599029,0.755984,75,-0.600423,0.755909,8.890510
68519,KXWONMC,0,-0.599029,0.755984,85,-0.600173,0.755861,7.317598
68535,KXWONMC,0,-0.599029,0.755984,159,-0.599865,0.756244,5.501271
165160,ZUGDKX7,0,-0.858956,-0.064697,265,-0.861073,-0.065147,13.616038
...,...,...,...,...,...,...,...,...
5885883,WRMXV6G,0,-0.609881,0.073220,575,-0.610692,0.072091,7.842377
5899134,0MV8YGD,0,0.053611,0.534504,157,0.055885,0.533995,14.842746
5901156,41HLHX5,1,-0.944483,0.082072,237,-0.943419,0.081702,6.919167
5925591,TI2FHS7,5,-0.496323,0.738555,841,-0.496138,0.740214,9.371573
